### This is a toy project. Do not take it serious.

Load the weather from CSV

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt


weather_df = pd.read_csv("./data/klarchiv_01975_daily_akt/produkt_klima_tag_20190904_20210306_01975.txt", 
                             delimiter=";", skipinitialspace=True, usecols=['TMK', 'RSK', 'SHK_TAG', 'SDK', 'MESS_DATUM', 'UPM'],
                        parse_dates=['MESS_DATUM'])
weather_df.rename(columns={'MESS_DATUM': 'date', 'RSK': 'precipitation', 'SDK': 'sun', 'SHK_TAG': 'snow', 'TMK': 'temperature', 'UPM': 'humidity'}, inplace=True)
weather_df.set_index('date',  inplace=True)


Load the incidence data from Excel

One row is messed up, I needed to fix the date there

In [2]:
#pd.set_option('display.max_rows', 100)

incidence_df = pd.read_excel("./data/d-inzidenz-zum-download-service.xlsx", usecols=['Unnamed: 1', 'Unnamed: 2'])
incidence_df.drop([0], inplace=True)
incidence_df.rename(columns={'Unnamed: 1': 'date', 'Unnamed: 2': 'cumulative_incidence'}, inplace=True)

incidence_df.loc[[316], 'date'] = '2021-01-21 00:00:00'  # messed up date in row 316
incidence_df['date'] = pd.to_datetime(incidence_df['date'])
incidence_df.set_index('date',  inplace=True)

incidence_df['daily_incidence'] = incidence_df['cumulative_incidence'].diff()
incidence_df.drop(['cumulative_incidence'], inplace=True, axis=1)


Load the Google mobility data from CSV

In [3]:
cols = ['sub_region_1', 'date', 'retail_and_recreation_percent_change_from_baseline', 
        'grocery_and_pharmacy_percent_change_from_baseline',
        'parks_percent_change_from_baseline',
        'transit_stations_percent_change_from_baseline',
        'workplaces_percent_change_from_baseline',
        'residential_percent_change_from_baseline']

mobility_df = pd.read_csv("./data/2020_DE_Region_Mobility_Report.csv", usecols=cols, parse_dates=['date'])
mobility_df = mobility_df.loc[mobility_df['sub_region_1'] == "Hamburg"]
mobility_df.reset_index(drop=True, inplace=True)
mobility_df.set_index('date',  inplace=True)


Join all the data frames to one big data frame

In [4]:
df = weather_df.join(incidence_df).join(mobility_df)
df.dropna(subset=['sub_region_1', 'daily_incidence'], inplace=True)


I think values where this seems appropriate should be scaled to a value between 0 and 1

I'm replacing the weather and mobility stuff but keep the original incidence cause this may be interesting to look at later.

In [5]:
import sklearn.preprocessing as prep

scaler = prep.MinMaxScaler()

scale_and_replace_cols = ['precipitation', 'temperature', 'humidity', 'sun', 'snow',
                          'grocery_and_pharmacy_percent_change_from_baseline',
                          'parks_percent_change_from_baseline',
                          'transit_stations_percent_change_from_baseline',
                          'workplaces_percent_change_from_baseline',
                          'residential_percent_change_from_baseline']
df[scale_and_replace_cols] = scaler.fit_transform(df[scale_and_replace_cols])

df[['daily_incidence_scaled']] = scaler.fit_transform(df[['daily_incidence']])


I think the best way to model this might be to take the incidence 7 days later as a target variable and then basically provide this to the model for training.


In [6]:
df['daily_incidence_scaled_target'] = df['daily_incidence_scaled'].shift(periods=-7)

Let's get a feel for the correlations

In [7]:
df.corrwith(df['daily_incidence_scaled_target'])

precipitation                                         0.020916
sun                                                  -0.558816
snow                                                  0.056000
temperature                                          -0.537001
humidity                                              0.579527
retail_and_recreation_percent_change_from_baseline   -0.265704
grocery_and_pharmacy_percent_change_from_baseline     0.008979
parks_percent_change_from_baseline                   -0.592633
transit_stations_percent_change_from_baseline        -0.290842
workplaces_percent_change_from_baseline              -0.137817
residential_percent_change_from_baseline              0.332857
daily_incidence_scaled                                0.890326
daily_incidence_scaled_target                         1.000000
dtype: float64

Let's split up the dataset into training and test data

In [8]:
from sklearn.model_selection import train_test_split


# not quit sure why the mobility data are sometimes NaN but I need to drop them to make the models work
df_only_complete = df.dropna(subset=['daily_incidence_scaled_target', 'parks_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline'])

predictive_cols = scale_and_replace_cols + ['daily_incidence_scaled']

X = df_only_complete[predictive_cols]
Y = df_only_complete['daily_incidence_scaled_target'].values

X_train, X_test, Y_train, Y_test = train_test_split (X, Y, test_size = 0.20, random_state=204408712) # using a randomly chosen but fixed random initializer to make things repoducable


Now we can benchmark various regression models (inspired by https://www.kaggle.com/junkal/selecting-the-best-regression-model)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor


pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
pipelines.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GBM', GradientBoostingRegressor())])))

results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='neg_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

ScaledLR: -0.011071 (0.004385)
ScaledLASSO: -0.057529 (0.012094)
ScaledEN: -0.057529 (0.012094)
ScaledKNN: -0.014302 (0.005175)
ScaledCART: -0.021635 (0.008154)
ScaledGBM: -0.011802 (0.004393)


Looks like linear regression works well?

In [39]:
# https://stackoverflow.com/a/53553226/1430384
def get_unscaled_incidence(value):
    colname = 'daily_incidence_scaled'
    exactmatch = df[df[colname] == value]
    if not exactmatch.empty:
        return exactmatch.index
    else:
        lowerneighbour_ind = df[df[colname] < value][colname].idxmax()
        return df.loc[lowerneighbour_ind]['daily_incidence']

# From https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html
# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(X_train, Y_train)

# df[predictive_cols].tail(5)
for row, result in zip(df[predictive_cols].tail(5).index  + pd.DateOffset(days=7), regr.predict(df[predictive_cols].tail(5))):
    print(f"Incidence on {row}: {get_unscaled_incidence(result)}")


Incidence on 2021-03-05 00:00:00: 188
Incidence on 2021-03-06 00:00:00: 206
Incidence on 2021-03-07 00:00:00: 147
Incidence on 2021-03-08 00:00:00: 258
Incidence on 2021-03-09 00:00:00: 195
